### i. Write code for k-nearest neighbors with Euclidean metric (or use a software package)

In [1]:
# util_notebook.py
import numpy as np

CLASS0 = "class 0"
CLASS1 = "class 1"
EPSILON = 1e-8

def load_data(file_path):
	"""read records from csv file"""
	row_data = np.genfromtxt(file_path, dtype=None, delimiter=',', encoding='utf-8')[1:]
	X = np.array(row_data[:, :-1], dtype=float)
	Y_row_str_data = row_data[:, row_data.shape[1] - 1]
	y = np.array([1 if x == "Abnormal" else 0 for x in Y_row_str_data])
	# print(X)
	# print(y)
	return X, y


def train_test_by_class_index(X, y, split_info_dict):
	"""According to the dict, split row data into X_train, X_test, y_train, y_test"""
	assert X.shape[0] == y.shape[0], \
		"the size of X must be equal to the size of y"
	assert split_info_dict is not None, \
		"Please initialize a dict with {CLASS0: int, CLASS1: int}"
	
	normal_indexes = (y == 0)
	abnormal_indexes = (y == 1)
	normal_limit = split_info_dict[CLASS0]
	abnormal_limit = split_info_dict[CLASS1]

	'''first 70 rows of class 0'''
	X_data_normal = X[normal_indexes, :][:normal_limit]
	y_data_normal = y[normal_indexes][:normal_limit]

	'''the rest of data in class 0'''
	X_rest_data_normal = X[normal_indexes, :][normal_limit:]
	y_rest_data_normal = y[normal_indexes][normal_limit:]

	'''first 140 rows of class 1'''
	X_data_abnormal = X[abnormal_indexes, :][:abnormal_limit]
	y_data_abnormal = y[abnormal_indexes][:abnormal_limit]
	'''the rest of data in class 1'''
	X_rest_data_abnormal = X[abnormal_indexes, :][abnormal_limit:]
	y_rest_data_abnormal = y[abnormal_indexes][abnormal_limit:]

	'''combine them into one array'''
	X_train = np.concatenate((X_data_normal, X_data_abnormal), axis=0)
	X_test = np.concatenate((X_rest_data_normal, X_rest_data_abnormal), axis=0)

	y_train = np.concatenate((y_data_normal, y_data_abnormal), axis=0)
	y_test = np.concatenate((y_rest_data_normal, y_rest_data_abnormal), axis=0)

	return X_train, X_test, y_train, y_test


In [2]:
def accuracy_score(y_true, y_predict):
	'''compare every single item in each array'''
	assert y_true.shape[0] == y_predict.shape[0], \
		"the shape of y_true need to be identical with y_predict"
	return sum(y_true == y_predict) / len(y_true)

In [3]:
# Aarons KNN algrithm based on Eulidean Distance
from math import sqrt
from collections import Counter

class KNeighborsClassifier:

	def __init__(self, n_neighbors):
		assert n_neighbors >= 1, "n_neighbors must be a valid number"
		self.n_neighbors = n_neighbors
		self._X_train = None
		self._y_train = None

	def fit(self, X_train, y_train):
		assert X_train.shape[0] == y_train.shape[0], ""
		assert self.n_neighbors <= X_train.shape[0], ""
		self._y_train = y_train
		self._X_train = X_train
		return self

	def predict(self, X_test):
		assert self._X_train is not None and self._y_train is not None, ""
		assert X_test.shape[1] == self._X_train.shape[1], ""

		y_predict = [self._predict(x) for x in X_test]
		return np.array(y_predict)

	def _predict(self, x):
		assert x.shape[0] == self._X_train.shape[1], ""
		# Euclidean Distance metric ==> sqrt(sum((x - y)^2))
		distances = [sqrt(np.sum(item - x) ** 2) for item in self._X_train]
		nearest = np.argsort(distances)
		topK_y = [self._y_train[i] for i in nearest[:self.n_neighbors]]
		votes = Counter(topK_y)
		return votes.most_common(1)[0][0]

	def score(self, X_test, y_test):
		y_predict = self.predict(X_test)
		return accuracy_score(y_test, y_predict)

	def __repr__(self):
		return "KNN(n_neighbors = %d)" % self.n_neighbors

In [4]:
# find the best k based on Aaron KNN algorithm
X_row_data, Y_row_data = load_data('./assets/data.csv')

split_info_dict = {CLASS0: 70, CLASS1: 140}

# Aaron's split method
X_train, X_test, y_train, y_test = train_test_by_class_index(X_row_data, Y_row_data, split_info_dict)
        
knn_clf = KNeighborsClassifier(n_neighbors=4)

knn_clf.fit(X_train, y_train)

knn_clf.score(X_test, y_test)

0.68